In [ ]:
import numpy as np
import random

In [35]:
# Example usage
questions = [
    "Explain the difference between a monolithic kernel and a microkernel. [0.7]", #1
    "Explain the difference between a process and a thread [0.5].", #2
    "How does virtual memory work in modern operating systems? [0.3]", #3
    "Describe the role of the kernel in an operating system. [0.4] ", #4
    "What are the various types of scheduling algorithms used in operating systems, and how do they differ? [1.5]", #5
    "How does a deadlock occur in an operating system, and what are some techniques to prevent it [1.2]?", #6
    "What is the purpose of an operating system [0.2]?", #7
    "What is the purpose of device drivers in an operating system? [0.6]", #8
    "Describe the boot process of a typical operating system. [0.5]", #9
    "How does an operating system manage file systems and disk storage? [1.0]", #10
    "What are the different types of file systems commonly used in operating systems, and how do they differ? [1.3]", #11
    "Explain the concept of process synchronization and why it's important in operating systems. [0.8]", #12
    "Discuss the role of interrupt handling in an operating system. [0.8]", #13
    "What is the difference between multiprogramming, multiprocessing, and multitasking? [1.8]", #14
    "How does an operating system handle memory management, including techniques like paging and segmentation? [0.9]" #15
]

In [36]:
def calculate_item_parameters():
    item_params = np.array([
        [0.8, 1], [0.5, 1], [0.3, 1], [0.4, 1], [1.5, 1],
        [1.2, 1], [0.2, 1], [0.6, 1], [0.5, 1], [1.0, 1],
        [1.3, 1], [0.8, 1], [0.8, 1], [1.8, 1], [0.8, 1]
    ])
    return item_params

In [54]:
def create_adaptive_algorithm(item_bank):
    ability_level = [0]
    responses = []
    used_indices = []

    def select_question():
        # Only used for the first question
        if not responses:
            starting_idx = random.randint(0, len(item_bank) - 1)
            used_indices.append(starting_idx)
            return item_bank[starting_idx], starting_idx

        item_params = calculate_item_parameters()
        adjusted_difficulty = ability_level[-1] - item_params[:, 0]
        item_probabilities = 1 / (1 + np.exp(-item_params[:, 1] * adjusted_difficulty))

        # Filter out used indices
        remaining_indices = [i for i in range(len(item_bank)) if i not in used_indices]

        if responses[-1] == "correct":
            # Filter out indices with item probability less than or equal to the previous item's probability
            remaining_indices = [i for i in remaining_indices if item_probabilities[i] <= item_probabilities[used_indices[-1]]]
            print(remaining_indices)
            if remaining_indices:
                next_question_index = max(remaining_indices, key=lambda x: item_probabilities[x])
            else:
                new_ls = [i for i in range(len(item_probabilities)) if item_probabilities[i] < item_probabilities[used_indices[-1]]]
                if len(new_ls) != 0:
                    next_question_index = max(new_ls, key=lambda x: item_probabilities[x])
                else:
                    new_ls = [i for i in range(len(item_probabilities)) if item_probabilities[i] >= item_probabilities[used_indices[-1]]]
                    next_question_index = max(new_ls, key=lambda x: item_probabilities[x])
        else:
            # Filter out indices with item probability greater than or equal to the previous item's probability
            remaining_indices = [i for i in remaining_indices if item_probabilities[i] >= item_probabilities[used_indices[-1]]]
            print(remaining_indices)
            if remaining_indices:
                next_question_index = min(remaining_indices, key=lambda x: item_probabilities[x])
            else:
                new_ls = [i for i in range(len(item_probabilities)) if item_probabilities[i] > item_probabilities[used_indices[-1]]]
                if len(new_ls) != 0:
                    next_question_index = min(new_ls, key=lambda x: item_probabilities[x])
                else:
                    new_ls = [i for i in range(len(item_probabilities)) if item_probabilities[i] <= item_probabilities[used_indices[-1]]]
                    next_question_index = min(new_ls, key=lambda x: item_probabilities[x])

        used_indices.append(next_question_index)
        return item_bank[next_question_index], next_question_index

    def update_ability_level(response, diff, disc):
        if response == "correct":
            val = ability_level[-1] + (1 * diff * disc)
        elif response == "incorrect":
            val = ability_level[-1] - (1 * diff * disc)
        ability_level.append(val)
        responses.append(response)
        print("Student Performance: ", ability_level)
        return ability_level

    return select_question, update_ability_level

In [61]:
select_question, update_ability_level = create_adaptive_algorithm(questions)

In [62]:
item_param = calculate_item_parameters()
ques, idx = select_question()
print("Q] ", ques)
response = int(input())
val = "correct" if response == 1 else "incorrect"
update_ability_level(val, item_param[idx][0], item_param[idx][1])

Q]  Explain the difference between a monolithic kernel and a microkernel. [0.7]
Student Performance:  [0, 0.8]


[0, 0.8]

In [63]:
for i in range(len(questions)-1):
    ques, idx = select_question()
    print("Q] ", ques)
    response = int(input())
    val = "correct" if response == 1 else "incorrect"
    ls = update_ability_level(val, item_param[idx][0], item_param[idx][1])

[4, 5, 9, 10, 11, 12, 13, 14]
Q]  Explain the concept of process synchronization and why it's important in operating systems. [0.8]
Student Performance:  [0, 0.8, 1.6]
[4, 5, 9, 10, 12, 13, 14]
Q]  Discuss the role of interrupt handling in an operating system. [0.8]
Student Performance:  [0, 0.8, 1.6, 0.8]
[1, 2, 3, 6, 7, 8, 14]
Q]  How does an operating system handle memory management, including techniques like paging and segmentation? [0.9]
Student Performance:  [0, 0.8, 1.6, 0.8, 0.0]
[1, 2, 3, 6, 7, 8]
Q]  What is the purpose of device drivers in an operating system? [0.6]
Student Performance:  [0, 0.8, 1.6, 0.8, 0.0, -0.6]
[1, 2, 3, 6, 8]
Q]  Explain the difference between a process and a thread [0.5].
Student Performance:  [0, 0.8, 1.6, 0.8, 0.0, -0.6, -1.1]
[2, 3, 6, 8]
Q]  Describe the boot process of a typical operating system. [0.5]
Student Performance:  [0, 0.8, 1.6, 0.8, 0.0, -0.6, -1.1, -1.6]
[2, 3, 6]
Q]  Describe the role of the kernel in an operating system. [0.4] 
Stud

In [64]:
def normalized_rating(ls):
    min_rating, max_rating = min(ls), max(ls)
    new = []
    for i in ls:
        val = (((i - min_rating) * 80) / (max_rating - min_rating)) + 20
        new.append(val)
    return new


In [65]:
student_ability = normalized_rating(ls)

In [66]:
student_ability

[68.78048780487805,
 84.39024390243902,
 100.0,
 84.39024390243902,
 68.78048780487805,
 57.07317073170732,
 47.31707317073171,
 37.5609756097561,
 29.75609756097561,
 23.902439024390247,
 20.0,
 55.1219512195122,
 59.024390243902445,
 78.53658536585365,
 55.1219512195122,
 35.609756097560975]